# Help functions and imports

## Imports

In [1]:
#all imported libraries are part of python standard standard library
import random
import math
import tkinter
import statistics as stats

## Various functions

Part of help functions relies on python pseudo random number generator.

- **dot_product** takes two vectors and returns their the dot_product
- **random_sample** takes three integers as input and returns the list of random unique integers in the asked interval
- **shuffle_list** takes a list and returns the list shuffled.

In [2]:
def dot_product(vector_a,vector_b):
    if len(vector_a)!=len(vector_b): 
        print ("Error: vector of different lengths")
        return False
    return sum([vector_a[i]*vector_b[i] for i in range(len(vector_a))])
 
#note that both lower and upper limit are included
def random_sample(lower_limit,upper_limit,sample_size):
    if sample_size > (upper_limit-lower_limit+1):
        print ("The asked sample size is to large")
    random_set=set([])
    random_list=[]
    while(len(random_set)<sample_size):
        a=random.randint(lower_limit,upper_limit)
        if a not in random_set:
            random_set.add(a)
            random_list.append(a)     
    return(random_list)

def shuffle_list(a_list):
    list_indices=list(range(len(a_list)))
    list_of_final_indices=[]
    for i in range(len(a_list)):
        current_choice=random.choice(list_indices)
        list_of_final_indices.append(current_choice)
        list_indices.remove(current_choice)
    return [a_list[i] for i in list_of_final_indices]
    

## Loading the data

**load_data_with_ones** takes a file with two comma separated numeric columns (coordinates on the real plane) and a second file with one numeric column.

The function returns list with four columns
- column 1 is made of ones for easier calculation of half step prediciton
- column 2 is the first column from the file with the data, $x_1$
- column 3 is the second column from the file with the data, $x_2$
- column 4 is the column with classes


In [3]:
# Also the csv library can be used.
def load_data_with_ones(training_data,training_class):

    train_data_file=open(training_data)
    train_data_file_lines=train_data_file.readlines()
    train_data_file.close()
    train_class_file=open(training_class)
    train_class_file_lines=train_class_file.readlines()
    train_class_file.close()
    data_list=[]
    for i in range(len(train_data_file_lines)):
        temp_list=[1]
        temp_line=train_data_file_lines[i].strip("\n")
        for j in train_data_file_lines[i].split(","):
            temp_list.append(j)
        temp_list=[float(i) for i in temp_list]
        temp_line=train_class_file_lines[i].strip("\n")
        temp_list.append(float(temp_line))
        data_list.append(temp_list)
    return data_list


# Implementation of the Perceptron algorithm with Stochastic Gradient Descent (SGD)

**unit_step_prediction** takes a single data point $(x_1,x_2)$ and a set of weights and returns predictions of its class

**learn_weigths** takes a train dataset, and two parameters, learning rate $\eta$ $(0<\eta<1)$ and epoch. It returns learned weigths.

**evaluation_metrics** takes a test dataset and learned weigths and returns the rate of success for the classification with the current weigths


In [4]:
def unit_step_prediction(data_row, weigths):
    z=dot_product(data_row,weigths)
    if z>=0:
        return 1
    else:
        return -1

def learn_weigths(data, epoch_num, learning_rate):
    weigths=[random.random()*random.choice([-1,1]) for i in range(len(data[0])-1)]
    for epoch in range(epoch_num):
        data_shuffled=shuffle_list(data)
        number_of_missclassified=0
        for data_row in data_shuffled:
            difference = unit_step_prediction(data_row[0:3],weigths)-data_row[3]
            if difference!=0:
                number_of_missclassified+=1
                for i in range(len(weigths)):
                    weigths[i]=weigths[i]-difference*data_row[i]*learning_rate
        #print("Epoch: %d" % (epoch+1), "Learning rate: %.2f" % learning_rate, "Success rate: %.2f %%" % ((1-number_of_missclassified/len(data))*100))
        if number_of_missclassified==0:
            data_shuffled.clear()
            return epoch+1,weigths
    data_shuffled.clear()
    return epoch+1,weigths

def evaluation_metrics(test_data,calculated_weigths):
    num_all=len(test_data)
    num_true=0
    for test_row in test_data:
        if test_row[3]==unit_step_prediction(test_row[0:3],calculated_weigths):
            num_true+=1
    return (num_true)/num_all*100

def perceptron(train_data,train_class,epoch, learning_rate):
    data=load_data_with_ones(train_data,train_class)
    final_epoch,learned_weigths=learn_weigths(data,epoch,learning_rate)
    evaluation=evaluation_metrics(data,learned_weigths)
    print("Evaluated success of the classifier is %.2f %%" % (evaluation), ", reached in",final_epoch, "epoch(s)", "with learning rate of", learning_rate )
    print("Learned weigths are:")
    print("w0 = ", "{0:.2f}".format(learned_weigths[0]))
    print("w1 = ", "{0:.2f}".format(learned_weigths[1]))
    print("w2 = ", "{0:.2f}".format(learned_weigths[2]))
    data.clear()
    return final_epoch,learned_weigths,evaluation

# The modification of the Perceptron for non linearly separable data

In perceptron for linearly separable data the classifier equation is $y=-w_1/w_2*x-w_0/w_2$, if we assume that the classifier is non_linear function with three parameters, we can imagine that the maybe it could be power function, exponential or logarithmic. Higher order polynomials would require more parameters.
The equation of implemented non linear classifier is: $y=-e^{w_1*x}/w_2-w_0/w_2$




**exponential_prediction** takes a single data point  (x1,x2)(x1,x2)  and a set of weights and returns predictions of its class

**learn_weigths** takes a train dataset, and two parameters, learning rate  ηη   (0<η<1)(0<η<1)  and epoch. It returns learned weigths.

**evaluation_metrics** takes a test dataset and learned weigths and returns the rate of success for the classification with the current weigths

In [174]:
def exponential_prediction(data_row, weigths):
    z=math.exp(data_row[1]*weigths[1])+weigths[0]+weigths[2]*data_row[2]
    if z>=0:
        return 1
    else:
        return -1

def learn_weigths_non_linear(data, epoch_num, learning_rate):
    weigths=[random.random()*random.choice([-1,1]) for i in range(len(data[0])-1)]
    for epoch1 in range(epoch_num):
        data_shuffled=shuffle_list(data)
        number_of_missclassified=0
        for data_row in data_shuffled:
            difference = exponential_prediction(data_row[0:3],weigths)-data_row[3]
            if difference!=0:
                number_of_missclassified+=1
                for i in range(len(weigths)):
                    if i==1:
                        weigths[i]=weigths[i]- difference/math.fabs(difference)*math.log(math.fabs(difference))*data_row[i]*learning_rate
                    else:
                        weigths[i]=weigths[i]-difference*data_row[i]*learning_rate
        #print("Epoch: %d" % (epoch+1), "Learning rate: %.2f" % learning_rate, "Success rate: %.2f %%" % ((1-number_of_missclassified/len(data))*100))
        if number_of_missclassified==0:
            data_shuffled.clear()
            return epoch1+1,weigths
    data_shuffled.clear()
    return epoch1+1,weigths

def evaluation_metrics_non_lin(test_data,calculated_weigths):
    num_all=len(test_data)
    num_true=0
    for test_row in test_data:
        if test_row[3]==exponential_prediction(test_row[0:3],calculated_weigths):
            num_true+=1
    return (num_true)/num_all*100


def perceptron_non_linear_modification(train_data,train_class,epoch, learning_rate):
    data=load_data_with_ones(train_data,train_class)
    final_epoch,learned_weigths=learn_weigths_non_linear(data,epoch,learning_rate)
    evaluation=evaluation_metrics_non_lin(data,learned_weigths)
    print("Evaluated success of the classifier is %.2f %%" % (evaluation), ", reached in",final_epoch, "epoch(s)", "with learning rate of", learning_rate )
    print("Learned weigths are:")
    print("w0 = ", "{0:.2f}".format(learned_weigths[0]))
    print("w1 = ", "{0:.2f}".format(learned_weigths[1]))
    print("w2 = ", "{0:.2f}".format(learned_weigths[2]))
    data.clear()
    return final_epoch,learned_weigths,evaluation

# The chart 

I hardcoded a chart using tkinter. I am used to relying on different tools such as matplotlib, gnuplot, matlab to make plots, it takes the learned weigths of both classifiers and other parameters and plots their equations on a real plane with samples.

In [93]:
def make_a_chart(linear_weigths,non_linear_weigts, epoch_lin,learning_rate_lin,epoch_non_lin,learning_rate_non_lin,eval_lin,eval_non_lin):
    
    #import data
    data_for_plot=load_data_with_ones("linsep-traindata.csv","linsep-trainclass.csv")
    data_for_non_lin_plot= load_data_with_ones("nonlinsep-traindata.csv","nonlinsep-trainclass.csv")
    #create window
    chart=tkinter.Tk()
    chart.title("Perceptron")
    #create canvas
    plots = tkinter.Canvas(chart, bg="white", height=600, width=1100)
    #add elements to canvas
    plots.create_rectangle(100,100,500,500,fill="light grey", outline="black")
    plots.create_rectangle(600,100,1000,500,fill="light grey", outline="black") 
    
    
    
    for data_line in data_for_plot:
        data_line[1]=int(data_line[1]*25+275)
        data_line[2]=int(data_line[2]*(-25)+325) 
        if data_line[3]==-1:
            plots.create_oval(data_line[1],data_line[2],data_line[1]+4,data_line[2]+4,fill="red")
        else:
            plots.create_oval(data_line[1],data_line[2],data_line[1]+4,data_line[2]+4,fill="blue")  
    
    for data_line in data_for_non_lin_plot:
        data_line[1]=int(data_line[1]*66.67+800)
        data_line[2]=int(data_line[2]*(-66.67)+333.33) 
        if data_line[3]==-1:
            plots.create_oval(data_line[1],data_line[2],data_line[1]+4,data_line[2]+4,fill="red")
        else:
            plots.create_oval(data_line[1],data_line[2],data_line[1]+4,data_line[2]+4,fill="blue")     
    
    w0= str("{0:.2f}".format(linear_weigths[0]))
    w1= str("{0:.2f}".format(linear_weigths[1]))
    w2= str("{0:.2f}".format(linear_weigths[2])) 
    lequation="Equation of linear classifier is y=-(" + w1+"x+" + w0 +")/"+w2
    
    line_x=[(-7 + i * 0.1) for i in range(160)]
    line_y=[(-linear_weigths[1]*x-linear_weigths[0])/linear_weigths[2] for x in line_x]
    for i in range(len(line_x)):
        if (line_y[i]<9) and (line_y[i]>-7):
            line_x[i]=int(line_x[i]*25+275)
            line_y[i]=int(line_y[i]*(-25)+325)
            plots.create_oval(line_x[i],line_y[i],line_x[i]+1,line_y[i]+1,fill="black")
    
    w1= str("{0:.2f}".format(non_linear_weigths[1]))
    w0= str("{0:.2f}".format(non_linear_weigths[0]))
    w2= str("{0:.2f}".format(non_linear_weigths[2]))
    nlequation="Equation of non linear classifier is y=-(exp(" + w1+"x)+" + w0 +")/"+w2
    
    exp_x=[(-2.95 + i * 0.05) for i in range(120)]
    exp_y=[]
    for j in range(len(exp_x)):
        exp_y.append(-(math.exp(non_linear_weigths[1]*exp_x[j])+non_linear_weigths[0])/non_linear_weigths[2])
          
    for i in range(len(exp_x)):
        if (exp_y[i]<3.5) and (exp_y[i]>-2.5):
            exp_x[i]=int(exp_x[i]*66.67+800)
            exp_y[i]=int(exp_y[i]*(-66.67)+333.33)
            plots.create_oval(exp_x[i],exp_y[i],exp_x[i]+1,exp_y[i]+1,fill="black")
    
    plots.create_text(90,100,text="9")
    plots.create_text(90,500,text="-7")
    plots.create_text(500,510,text="9")
    plots.create_text(100,510,text="-7")
    plots.create_text(590,100,text="3")
    plots.create_text(580,500,text="-2.5")
    plots.create_text(600,510,text="-3")
    plots.create_text(1000,510,text="3")
    plots.create_text(135,110,text= "Class -1", fill="red")
    plots.create_text(635,110,text= "Class -1", fill="red")
    plots.create_text(135,490,text= "Class 1", fill="blue")
    plots.create_text(635,490,text= "Class 1", fill="blue")
    plots.create_text(300,80,text=lequation)
    plots.create_text(800,80,text=nlequation)
    plots.create_text(300,60,text=("Number of epochs needed: "+ str(epoch_lin)))
    plots.create_text(800,60,text=("Number of epochs needed: "+ str(epoch_non_lin)))
    plots.create_text(300,40,text=("Learning rate: "+ str(learning_rate_lin)))
    plots.create_text(800,40,text=("Learning rate: "+ str(learning_rate_non_lin)))
    plots.create_text(300,540,text=("Evaluated quality of the classifier: "+ str(eval_lin) + " %"))
    plots.create_text(800,540,text=("Evaluated quality of the classifier: "+ str(eval_non_lin)+ " %"))

    plots.pack()
    tkinter.Button(chart, text="Bye", command=chart.destroy).pack()                 
    chart.mainloop()
    data_for_plot.clear()
    data_for_non_lin_plot.clear()
    line_x.clear()
    line_y.clear()
    exp_x.clear()
    exp_y.clear()
    return

# The execution and the model (run and change parameters here)

In [179]:
#change parameters if you want (learning rates should be in the (0,1) interval) and epoch should be positive integer

#parameters for the perceptron function
max_epoch_lin=100
learning_rate_lin=0.1
#parameters for the modified perceptron function
max_epoch_non_lin=100
learning_rate_non_lin=0.1

In [187]:
#execution of the program
#all cells above should be executed first
final_epoch_lin,linear_weigths,evaluation_lin=perceptron("linsep-traindata.csv","linsep-trainclass.csv",max_epoch_lin,learning_rate_lin)
final_epoch_non_lin,non_linear_weigths,evaluation_non_lin=perceptron_non_linear_modification("nonlinsep-traindata.csv","nonlinsep-trainclass.csv",max_epoch_non_lin,learning_rate_non_lin)
make_a_chart(linear_weigths,non_linear_weigths,final_epoch_lin,learning_rate_lin,final_epoch_non_lin,learning_rate_non_lin,evaluation_lin,evaluation_non_lin)

Evaluated success of the classifier is 100.00 % , reached in 2 epoch(s) with learning rate of 0.1
Learned weigths are:
w0 =  -0.72
w1 =  -0.81
w2 =  -0.81
Evaluated success of the classifier is 100.00 % , reached in 15 epoch(s) with learning rate of 0.1
Learned weigths are:
w0 =  -1.90
w1 =  -0.47
w2 =  -1.25


# Some properties of the training set

In [80]:
data=load_data_with_ones("linsep-traindata.csv","linsep-trainclass.csv")
list_of_classes=[data[i][3] for i in range(len(data))]
list_of_x_neg=[data[i][1] for i in range(len(data)) if data[i][3]==-1 ]
list_of_y_neg=[data[i][2] for i in range(len(data)) if data[i][3]==-1]
list_of_x_pos=[data[i][1] for i in range(len(data)) if data[i][3]==1]
list_of_y_pos=[data[i][2] for i in range(len(data)) if data[i][3]==1]
print("Properties of the linearly separable data")
print("Training sample is composed of equal number of each class, mean value of the classes of the whole sample is:",stats.mean(list_of_classes))
print("Mean and standard deviation of the sample")
print("Class -1, coordinate x:",stats.mean(list_of_x_neg),stats.stdev(list_of_x_neg))
print("Class -1, coordinate y:",stats.mean(list_of_y_neg),stats.stdev(list_of_y_neg))
print("Class  1, coordinate x:",stats.mean(list_of_x_pos),stats.stdev(list_of_x_pos))
print("Class  1, coordinate y:",stats.mean(list_of_y_pos),stats.stdev(list_of_y_pos))
list_of_classes.clear()
list_of_x_neg.clear()
list_of_x_pos.clear()
list_of_y_neg.clear()
list_of_y_pos.clear()
list_of_classes.clear()

Properties of the linearly separable data
Training sample is composed of equal number of each class, mean value of the classes of the whole sample is: 0.0
Mean and standard deviation of the sample
Class -1, coordinate x: 3.49382432 2.244669569682184
Class -1, coordinate y: 3.7950998 2.1103647361284277
Class  1, coordinate x: -3.390084 1.0274804558169046
Class  1, coordinate y: -3.547274 0.9364283732831871


In [81]:
data=load_data_with_ones("nonlinsep-traindata.csv","nonlinsep-trainclass.csv")
list_of_classes=[data[i][3] for i in range(len(data))]
list_of_x_neg=[data[i][1] for i in range(len(data)) if data[i][3]==-1 ]
list_of_y_neg=[data[i][2] for i in range(len(data)) if data[i][3]==-1]
list_of_x_pos=[data[i][1] for i in range(len(data)) if data[i][3]==1]
list_of_y_pos=[data[i][2] for i in range(len(data)) if data[i][3]==1]
print("Properties of the linearly non separable data")
print("Training sample is composed of equal number of each class, mean value of the classes of the whole sample is:",stats.mean(list_of_classes))
print("Mean and standard deviation of the sample")
print("Class -1, coordinate x:",stats.mean(list_of_x_neg),stats.stdev(list_of_x_neg))
print("Class -1, coordinate y:",stats.mean(list_of_y_neg),stats.stdev(list_of_y_neg))
print("Class  1, coordinate x:",stats.mean(list_of_x_pos),stats.stdev(list_of_x_pos))
print("Class  1, coordinate y:",stats.mean(list_of_y_pos),stats.stdev(list_of_y_pos))
list_of_classes.clear()
list_of_x_neg.clear()
list_of_x_pos.clear()
list_of_y_neg.clear()
list_of_y_pos.clear()
list_of_classes.clear()

Properties of the linearly non separable data
Training sample is composed of equal number of each class, mean value of the classes of the whole sample is: 0.0
Mean and standard deviation of the sample
Class -1, coordinate x: 0.0467366 1.068616852383854
Class -1, coordinate y: -0.24302462 0.7617860774735961
Class  1, coordinate x: 0.165596864 1.0377973396158682
Class  1, coordinate y: -1.0166386 0.30543693838606534


# Comments and questions


- Should the provided sample be split into the train sample and the test sample, or the test sample should be subset of the train sample? In my opinion 
- Should the sample be shuffled in each epoch? Yes. There is the option to choose a part sample for each epoch, and from my point of view as the sample is not very large, and it doesn't make a big impact on the computing time to use the whole sample, shuffling is enough. Also choosing a smaller random sample from the balanced sample (sum of classes==0), increses chances that the sample is unbalanced in regard of classes
- Is it enough to evaluate the function with a simple ratio success/all (classification accuracy)? In my opinion yes, as the sample is balanced. 
- The whole sample is used for the evaluation of the classifier
- The calculated weigths will be always a bit different as the initial weigths are random, and also because the data is being shuffled. 
- All libraries I used are part of standard python librabry, I understood that only libraries I can't use are pip libraries
- The representation of the data and classifier is not completely accurate as the coordinate system is made of a descrete number of points, and the preented values are rounded.
- The modelling ends as soon as it reaches 0 missclassifications, as the whoe sample is used in each epoch there no chance that there will be a missclassification in the next epoch. Had I used the random subsample in each epoch I would prefere to leave it to run until it reaches the given epoch. Especially if lower percentage of sample is used in each epoch, or if the standard deviation of the sample is high.
- The non linear modification of the perceptron will converge only for small learning rates (<0.4)

# References


1. https://sebastianraschka.com/Articles/2015_singlelayer_neurons.html#the-unit-step-function